# Entrenamiento y evaluación modelo de reconocimiento de caras con Tensorflow

La idea de esta notebook es realizar el entrenamiento y evaluación de un modelo mediane tensorflow y keras.

Las imagenes para ajustar y evaluar los modelos son las mismas de la notebook "01_entrenar_modelo\01_entrenar_modelo.ipynb".

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import os
from PIL import Image
import cv2
import re    

## Define los parámetros de ejecución de la notebook

In [2]:
# parametros notebook
DIM = 30
seed = 42
np.random.seed(42)
tf.random.set_seed(42)

## Define los modelos a entrenar

In [3]:
# planteamos distintas pruebas

#n_pca: cantidad de componentes principales a considerar
#excl_n_prim_comp: excluir las primeras excl_n_prim_comp componenetes principales (0 no se excluye ninguna)
#nueronas_layer_1: neuronas capa oculta 1
#nueronas_layer_2: neuronas capa oculta 2
#n_epochs: cantidad de epochs... NO SE TOMA EN CUENTA... SE HARDCODEA MÁS ABAJO!!!

# PARA LA ULTIMA PARTE DE LA NOTEBOOK ES IMPORTANTE TENER A MANO LOS PARAMETROS
# "n_pca" y "excl_n_prim_comp" del modelo seleccionado!!!

pruebas = {
    "prueba_1": {"n_pca":60, "excl_n_prim_comp":5, "nueronas_layer_1":40, "nueronas_layer_2":25, "n_epochs":300},
    "prueba_2": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":300},
    "prueba_3": {"n_pca":100, "excl_n_prim_comp":0, "nueronas_layer_1":45, "nueronas_layer_2":26, "n_epochs":200},
    "prueba_4": {"n_pca":100, "excl_n_prim_comp":3, "nueronas_layer_1":60, "nueronas_layer_2":30, "n_epochs":200},
}

In [4]:
# Agrego esto para realizar algunas pruebas en otro directorio pero utilizando las funciones definidas por fede...
import os
import sys

# Obtener el path de ejecucion de la notebook
current_dir = os.getcwd()

# Define el nombre del proyecto
root_dir_name = "NuestrasCaras"

# Obtiene el path del proyecto
while not os.path.basename(current_dir) == root_dir_name:
    current_dir = os.path.dirname(current_dir)
    
# Agrega path a librerias
sys.path.append(current_dir)

## Leer imágenes para entrenamiento

In [5]:
dim_imagenes = DIM
data_dir = "01_entrenar_modelo/fotos_entrenamiento"
nombres = []
imagenes = []

# Cargar imágenes y etiquetas
dir = os.path.join(current_dir,"01_entrenar_modelo","fotos_entrenamiento")
for archivo in os.listdir(dir):
    if archivo.endswith('.jpeg') or archivo.endswith('.jpg'):
        nombre = archivo.split('_')[0].replace(".jpg","").replace(".jpeg","")
        nombre = re.sub(r"\d+", "", nombre)
        ruta_imagen = os.path.join(dir, archivo)
        imagen = Image.open(ruta_imagen)
        imagen = np.array(imagen.resize((dim_imagenes, dim_imagenes)))  # Redimensionar imágenes para un tamaño uniforme
        if len(imagen.shape) == 3 and imagen.shape[2] == 3:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises si es necesario
        imagenes.append(imagen.flatten())
        nombres.append(nombre)

imagenes_all = np.array(imagenes)
nombres_all = np.array(nombres)

# estand
imagenes_all = imagenes_all/255.0

In [6]:
print(imagenes_all.shape)
print(nombres_all.shape)


(10459, 900)
(10459,)


## Leer imagenes para testeo

In [7]:
data_dir = ("01_entrenar_modelo/fotos_test_entrenamiento")
nombres = []
imagenes = []

# Cargar imágenes y etiquetas
dir = os.path.join(current_dir,data_dir)
for archivo in os.listdir(dir):
    if archivo.endswith('.jpeg') or archivo.endswith('.jpg'):
        nombre = archivo.split('_')[0].replace(".jpg","").replace(".jpeg","")
        nombre = re.sub(r"\d+", "", nombre)
        ruta_imagen = os.path.join(dir, archivo)
        imagen = Image.open(ruta_imagen)
        imagen = np.array(imagen.resize((dim_imagenes, dim_imagenes)))  # Redimensionar imágenes para un tamaño uniforme
        if len(imagen.shape) == 3 and imagen.shape[2] == 3:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises si es necesario
        imagenes.append(imagen.flatten())
        nombres.append(nombre)

new_images = np.array(imagenes)
nombres_new = np.array(nombres)

# estand
new_images = new_images/255.0

In [8]:
print(new_images.shape)
print(nombres_new.shape)


(30, 900)
(30,)


## Entrenamiento y testeo del modelo

In [9]:
# define funcion para entrenar 
def run_keras_model(n_pca, excl_n_prim_comp, nueronas_layer_1, nueronas_layer_2, n_epochs, imagenes, nombres, nuevas_imagenes, nuevos_nombres):
    
    # Dividir en entrenamiento y prueba
    # se comentan ya que no se divide la base en train y test...
    # se va a utilizar TODAS las imagenes para el entrenamiento...
    # tenemos imagenes extras de todos los integrantes para realizar la evaluacion...
    #X_train, X_test, y_train, y_test = train_test_split(imagenes, nombres, test_size=0.05, random_state=42, stratify=nombres)
    X_train = imagenes #np.array(imagenes)
    y_train = nombres #np.array(nombres).reshape(-1)
    
    # Aplicar PCA
    pca = PCA(n_components=n_pca, random_state=seed)
    X_train_pca = pca.fit_transform(X_train)
    #X_test_pca = pca.transform(X_test)
    
    # Escalar los datos 
    scaler = StandardScaler()
    #scaler = MinMaxScaler() # en caso de ser necesario probar con otro metodo de escalado
    X_train_pca_scaled = scaler.fit_transform(X_train_pca)
    #X_test_pca_scaled = scaler.transform(X_test_pca)
    
    
    # Codificar las etiquetas
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    #y_test_encoded = encoder.transform(y_test)
    y_train_categorical = to_categorical(y_train_encoded, num_classes=19)
    #y_test_categorical = to_categorical(y_test_encoded, num_classes=19)
    
    
    # Definir la red neuronal
    entreno_con = X_train_pca_scaled[:,excl_n_prim_comp:]
    #testeo_con = X_test_pca_scaled[:,excl_n_prim_comp:]
    model = Sequential()
    model.add(Input(shape=entreno_con.shape[1:]))  # Definir la entrada del modelo
    model.add(Dense(nueronas_layer_1, activation='sigmoid'))
    model.add(Dense(nueronas_layer_2, activation='sigmoid'))
    model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

    # Compilar el modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo
    model.fit(entreno_con, y_train_categorical, epochs=n_epochs, batch_size=10, validation_split=0.3)

    # Evaluar el modelo
    #loss, accuracy = model.evaluate(testeo_con, y_test_categorical)
    #print(f'Precisión en el conjunto de prueba: {accuracy * 100:.2f}%')
    
    
    # PREDICCIONES
    # 2. Aplicar PCA
    new_images_pca = pca.transform(nuevas_imagenes)
    new_images_pca_scaled = scaler.transform(new_images_pca)
    evaluo_con = new_images_pca_scaled[:,excl_n_prim_comp:]

    # 3. Hacer predicciones
    predictions = model.predict(evaluo_con)

    # Obtener los nombres correspondientes a las clases predichas
    predicted_names = encoder.inverse_transform(np.argmax(predictions, axis=1))

    # Imprimir las predicciones
    total_predict = len(nuevos_nombres)
    total_correcto = 0
    for real, pred in zip(nuevos_nombres, predicted_names):
        if real == pred:
            total_correcto += 1
    
    # devuelve resultado corrida
    resultado = total_correcto/total_predict
    return resultado, model

In [10]:
resutados = {}
modelos = {}
for prueba, params in pruebas.items():
    print(f"{prueba}: ")
    resultado, modelo = run_keras_model(n_pca=params.get('n_pca'), 
                    excl_n_prim_comp=params.get('excl_n_prim_comp'), 
                    nueronas_layer_1=params.get('nueronas_layer_1'),
                    nueronas_layer_2=params.get('nueronas_layer_2'),
                    n_epochs=params.get('n_epochs'),
                    imagenes=imagenes_all,
                    nombres=nombres_all,
                    nuevas_imagenes=new_images,
                    nuevos_nombres=nombres_new
                    )
    resutados[prueba]=resultado
    modelos[prueba]=modelo
resutados

prueba_1: 
Epoch 1/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1850 - loss: 2.6047 - val_accuracy: 0.0000e+00 - val_loss: 6.1539
Epoch 2/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5327 - loss: 1.7958 - val_accuracy: 0.0000e+00 - val_loss: 7.1926
Epoch 3/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7124 - loss: 1.2393 - val_accuracy: 0.0000e+00 - val_loss: 7.9184
Epoch 4/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7778 - loss: 0.9255 - val_accuracy: 0.0000e+00 - val_loss: 8.4674
Epoch 5/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8205 - loss: 0.7221 - val_accuracy: 0.0000e+00 - val_loss: 8.9302
Epoch 6/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8494 - loss: 0.5870 - val_accuracy: 0.0019 - val_loss: 9.3494
Epoch 7/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8761 - loss: 0.4918 - val_accuracy: 0.0073 - val_loss: 9.7394
Epoch 8/300
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0

{'prueba_1': 0.4666666666666667,
 'prueba_2': 0.6,
 'prueba_3': 0.7,
 'prueba_4': 0.5333333333333333}

## Segundo testeo del modelo con nuevas imagenes del equipo "Grupo 1"

In [11]:
data_dir = ("02_probar_nuevas_fotos/fotos_prueba_recortadas")
nombres = []
imagenes = []

# Cargar imágenes y etiquetas
dir = os.path.join(current_dir,data_dir)
for archivo in os.listdir(dir):
    if archivo.endswith('.jpeg') or archivo.endswith('.jpg'):
        nombre = archivo.split('_')[0].replace(".jpg","").replace(".jpeg","")
        nombre = re.sub(r"\d+", "", nombre)
        ruta_imagen = os.path.join(dir, archivo)
        imagen = Image.open(ruta_imagen)
        imagen = np.array(imagen.resize((dim_imagenes, dim_imagenes)))  # Redimensionar imágenes para un tamaño uniforme
        if len(imagen.shape) == 3 and imagen.shape[2] == 3:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises si es necesario
        imagenes.append(imagen.flatten())
        nombres.append(nombre)

new_images = np.array(imagenes)
nombres_new = np.array(nombres)

# estand
new_images = new_images/255.0
print(new_images.shape)
print(nombres_new.shape)


(6, 900)
(6,)


In [12]:
len(nombres_new)

6

In [13]:
# Define funcion para las predicciones
def get_preds_predict2(model, n_pca, excl_n_prim_comp, imagenes, nombres, nuevas_imagenes, nuevos_nombres):
    
    # Dividir en entrenamiento y prueba
    #X_train, X_test, y_train, y_test = train_test_split(imagenes, nombres, test_size=0.05, random_state=42, stratify=nombres)
    X_train = imagenes#np.array(imagenes)
    y_train = nombres#np.array(nombres)

    # Aplicar PCA
    pca = PCA(n_components=n_pca, random_state=12)
    X_train_pca = pca.fit_transform(X_train)
    #X_test_pca = pca.transform(X_test)
    
    # Escalar los datos 
    scaler = StandardScaler()
    #scaler = MinMaxScaler()
    X_train_pca_scaled = scaler.fit_transform(X_train_pca)
    #X_test_pca_scaled = scaler.transform(X_test_pca)
    
    
    # Codificar las etiquetas
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    #y_test_encoded = encoder.transform(y_test)
    #y_train_categorical = to_categorical(y_train_encoded, num_classes=19)
    #y_test_categorical = to_categorical(y_test_encoded, num_classes=18)
    
    
    # Definir la red neuronal
    #entreno_con = X_train_pca_scaled[:,excl_n_prim_comp:]
    #testeo_con = X_test_pca_scaled[:,excl_n_prim_comp:]    
    
    # PREDICCIONES
    # 2. Aplicar PCA
    new_images_pca = pca.transform(nuevas_imagenes)
    new_images_pca_scaled = scaler.transform(new_images_pca)
    evaluo_con = new_images_pca_scaled[:,excl_n_prim_comp:]

    # 3. Hacer predicciones
    predictions = model.predict(evaluo_con)

    # Obtener los nombres correspondientes a las clases predichas
    predicted_names = encoder.inverse_transform(np.argmax(predictions, axis=1))

    # Imprimir las predicciones
    total_predict = len(nuevos_nombres)
    total_correcto = 0
    for real, pred in zip(nuevos_nombres, predicted_names):
        if real == pred:
            total_correcto += 1
         
    # df predicciones        
    df_preds = pd.DataFrame(predictions.round(2))
    class_names = encoder.classes_
    df_preds.columns = class_names
    
    # devuelve resultado corrida
    resultado = total_correcto/total_predict
    return resultado, df_preds, predictions

In [14]:
# SOLO SE MODIFICA MANUALMENTE EL MODELO SELECCIONADO!
prueba_seleccionada = "prueba_1"

modelo_seleccionado = modelos[prueba_seleccionada]

# Obtener resultados
n_pca_modelo_seleccionado = pruebas[prueba_seleccionada].get("n_pca")
excl_n_prim_comp_modelo_seleccionado = pruebas[prueba_seleccionada].get("excl_n_prim_comp")
resultado, df_preds, predicted_names = get_preds_predict2(
    model=modelo_seleccionado, 
    n_pca=n_pca_modelo_seleccionado, 
    excl_n_prim_comp=excl_n_prim_comp_modelo_seleccionado, 
    imagenes=imagenes_all, 
    nombres=nombres_all, 
    nuevas_imagenes=new_images, 
    nuevos_nombres=nombres_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [15]:
# Proporción de aciertos
print("Proporción de aciertos:")
resultado

Proporción de aciertos:


0.5

In [16]:
# Predicciones
print("Predicciones (cada fila es una foto):")
df_preds.index = nombres_new
df_preds

Predicciones (cada fila es una foto):


,abel,carlos,federicoG,federicoR,florencia,francoA,francoS,gerard,gustavo,joaquin,juan,lautaro,lisandro,marco,matias,natalia,noelia,paola,victorio
lautaro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lautaro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lautaro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lautaro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
paola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
paola,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
